## Zadania: Retrieval Augmented Generation RAG


Zadanie 1 \
1. Przeanalizuj poniższy kod wykorzystujacy bazę wektorową oraz RAG.
2. Podmień dokumenty źródłowe oraz tekst zapytania w ostatniej linii.

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

# Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Dokumenty źródłowe
docs = [ # Podmień dokumenty źródłowe na swoje
    "LangChain to framework do pracy z LLM.",
    "RAG łączy dopasowanie kontekstu z generacją odpowiedzi.",
    "FAISS to biblioteka do przechowywania i wyszukiwania embeddingów."
]

# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splits = splitter.create_documents(docs)

# Embeddings + vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiadaj tylko na podstawie kontekstu:\n{context}"),
    ("user", "{question}")
])

# Pipeline
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)


print(rag_chain.invoke({"question": "Co to jest FAISS?"})) # <- Zmień pytanie


FAISS to biblioteka do przechowywania i przeszukiwania dużych zbiorów wektorów.
